<a href="https://colab.research.google.com/github/tpmmthomas/Sepsis-diagnosis-from-pairwise-single-cell-RNA-Continued/blob/master/NEW_NEW_capsnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras import initializers, layers
from datetime import date
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.python.client import device_lib
from tensorflow.keras import callbacks
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from trans import *
from cap import *

2024-07-04 10:25:44.220696: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
xtrain = r"../data/dataSC/NEW_training_sample.csv.gz"
ytrain = r"../data/dataSC/NEW_training_label.csv.gz"
xtest = r"../data/dataSC/NEW_testing_sample.csv.gz"
ytest = r"../data/dataSC/NEW_testing_label.csv.gz" 

i=0
samplesdf = pd.read_csv(xtrain,compression ="gzip",delimiter=',')
x_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytrain,compression ="gzip",delimiter=',')
y_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(xtest,compression ="gzip",delimiter=',')
x_test = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytest,compression ="gzip",delimiter=',')
y_test = samplesdf.to_numpy()

print("done")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

done
(38483, 3030)
(38483, 1)
(4265, 3030)
(4265, 1)


In [4]:
x_train.max(),x_train.min()

(9.09, 0.0)

In [3]:
common_indicator = pd.read_csv('../data/dataBulk/common_rna_indicator.csv',index_col=0).values.squeeze().tolist()
x_train = x_train.T[common_indicator].T
print(x_train.shape)
x_test = x_test.T[common_indicator].T
print(x_test.shape)

(38483, 2869)
(4265, 2869)


In [5]:
x_test.max(), x_test.min()

(8.84, 0.0)

In [4]:
x_train_tf = tf.reshape(x_train,(len(x_train),2869,1))
print(x_train_tf.shape)

2024-07-01 01:18:31.662550: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-01 01:18:31.665022: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-07-01 01:18:31.710802: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 715.34GiB/s
2024-07-01 01:18:31.711025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:73:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.67GiB deviceMemoryBandwidth: 715.34GiB/s
2024-07-01 01:18:31.711056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-07-01 

(38483, 2869, 1)


In [5]:
def CapsNet(input_shape, n_class, routings):
    inputs = layers.Input(shape=input_shape)
    primarycaps = PrimaryCap(inputs, dim_capsule=8, n_channels=1, kernel_size=1, strides=1, padding='valid')
    groupcaps,routing_weight = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='groupcaps')(primarycaps)

    transformer_block = TransformerBlock(embed_dim=16, num_heads=2, ff_dim=128)
    x = transformer_block(groupcaps)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(150, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)
    return model


def margin_loss(y_true, y_pred):
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))
    return K.mean(K.sum(L, 1))


def test(model, X_test, y_test, args):
    y_pred, x_recon = model.predict(X_test, batch_size=20)
    print('-'*30 + 'Begin: test' + '-'*30)
    print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])
    return y_pred

In [6]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

aucs = []
auprcs = []

for fold, (train_index, test_index) in enumerate(skf.split(x_train_tf, y_train)):
    print(f"Training fold {fold+1}...")
    
    x_train_fold = tf.gather(x_train_tf, train_index)
    x_val_fold = tf.gather(x_train_tf, test_index)
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
    # define model
    model = CapsNet(input_shape=x_train_tf[1,:,:].shape, n_class=10, routings=5)
    model.compile(optimizer=optimizers.Adam(lr=0.0001, amsgrad=True),
                  loss='binary_crossentropy', metrics=["accuracy"])
    
    earlystopping = callbacks.EarlyStopping(monitor="val_accuracy", 
                                            mode="min", patience=5, 
                                            restore_best_weights=True)
    
    history = model.fit(x_train_fold, y_train_fold, batch_size=64, epochs=20,
                        validation_data=(x_val_fold, y_val_fold),
                        shuffle=True, callbacks=[earlystopping])
    
    # 
    y_pred_prob = model.predict(x_val_fold).ravel()
    
    # 
    fpr, tpr, _ = roc_curve(y_val_fold, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
    # 
    precision, recall, _ = precision_recall_curve(y_val_fold, y_pred_prob)
    auprc = average_precision_score(y_val_fold, y_pred_prob)
    auprcs.append(auprc)
    
    print(f"Fold {fold+1} AUC: {roc_auc:.5f}, AUPRC: {auprc:.5f}")
    
    # 
    today = date.today()
    # model.save(f'./model/weightmodel_noindv_round{fold+1}_{today}')
    


Training fold 1...


2024-07-01 01:18:45.011626: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-07-01 01:18:45.012082: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


Epoch 1/20


2024-07-01 01:18:47.608868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


482/482 [==============================] - 97s 195ms/step - loss: 0.6436 - accuracy: 0.6117 - val_loss: 0.5342 - val_accuracy: 0.7370
Epoch 2/20
482/482 [==============================] - 95s 197ms/step - loss: 0.4408 - accuracy: 0.7970 - val_loss: 0.4200 - val_accuracy: 0.8073
Epoch 3/20
482/482 [==============================] - 95s 197ms/step - loss: 0.3971 - accuracy: 0.8196 - val_loss: 0.4144 - val_accuracy: 0.8098
Epoch 4/20
482/482 [==============================] - 95s 198ms/step - loss: 0.3576 - accuracy: 0.8393 - val_loss: 0.4197 - val_accuracy: 0.8028
Epoch 5/20
482/482 [==============================] - 95s 196ms/step - loss: 0.3465 - accuracy: 0.8411 - val_loss: 0.3932 - val_accuracy: 0.8156
Epoch 6/20
482/482 [==============================] - 95s 197ms/step - loss: 0.3263 - accuracy: 0.8504 - val_loss: 0.3893 - val_accuracy: 0.8207
Fold 1 AUC: 0.87291, AUPRC: 0.87455
Training fold 2...
Epoch 1/20
482/482 [==============================] - 98s 198ms/step - loss: 0.6492 - 

In [7]:
aucs, auprcs

([0.872914926936048,
  0.8951951785014336,
  0.868325487131336,
  0.8896985533257314,
  0.8805772802810565],
 [0.8745503897108097,
  0.8946477478497267,
  0.864737088710366,
  0.889097658664193,
  0.8741331142605231])

In [8]:
# 
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve - Fold {fold+1}')
plt.legend(loc='lower right')
#plt.savefig(f'roc_curve_fold{fold+1}_{today}.png')
plt.close()

plt.figure()
plt.plot(recall, precision, label=f'PR curve (AUPRC = {auprc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve - Fold {fold+1}')
plt.legend(loc='lower left')
#plt.savefig(f'pr_curve_fold{fold+1}_{today}.png')
plt.close()

In [8]:
testresult=model.predict(x_test)
i = 0
correct = 0
for x in testresult:
    if x >=0.5 and y_test[i] == 1:
        correct = correct + 1
    elif x < 0.5 and y_test[i] == 0:
        correct = correct + 1
    i = i + 1
testacc = correct/i
print("Testing accuracy:",testacc)

fpr, tpr, _ = roc_curve(y_test,testresult)
roc_auc = auc(fpr,tpr)
print("AUROC = %.05f"% roc_auc)

precision, recall, _ = precision_recall_curve(y_test,testresult)
prc_auc = auc(recall,precision)
print("AUPRC = %.05f"% prc_auc)
ss = np.zeros((len(testresult)))
i = 0
for x in testresult:
    if x >= 0.5:
        ss[i] = 1
    else:
        ss[i] = 0
    i = i + 1
f1s = f1_score(y_test,ss)
print("f1_score = %.05f"% f1s)

Testing accuracy: 0.7798358733880422
AUROC = 0.87532
AUPRC = 0.86604
f1_score = 0.74967


In [12]:
# pd.DataFrame(testresult).to_csv('../capsnetweighted_rount5.csv',header=None, index=None)
# pd.DataFrame(y_test).to_csv('../capsnet_y_rount5.csv',header=None, index=None)